In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import sklearn.metrics as sm
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

pd.options.display.max_rows = 1500
pd.options.display.max_columns = 100

In [ ]:
#read data
train_set = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_set = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
print(len(train_set))
print(len(test_set))

In [ ]:
train_set.head()

In [ ]:
train_set.info()

In [ ]:
print(len(train_set.columns))

In [ ]:
print(train_set.isnull().sum().sort_values(ascending = False))


**Columns with null values**

* PoolQC           1453
* MiscFeature      1406
* Alley            1369
* Fence            1179
* FireplaceQu       690
* LotFrontage       259
* GarageType         81
* GarageYrBlt        81
* GarageFinish       81
* GarageQual         81
* GarageCond         81
* BsmtExposure       38
* BsmtFinType2       38
* BsmtQual           37
* BsmtCond           37
* BsmtFinType1       37


PoolQC, MicsFeature, Alley, Fence, have lot of null data so I will remove those columns from dataset. 


In [ ]:
train_set=train_set.drop(columns=['PoolQC', 'MiscFeature', 'Alley', 'Fence'])
train_set.describe()

In [ ]:
print(len(train_set.columns))

**Replace Null Values**

**Categorical**
* BsmtFinType2       38
* BsmtExposure       38
* BsmtQual           37
* BsmtCond           37
* BsmtFinType1       37
* MasVnrType          8
* Electrical          1
* FireplaceQu       690
* GarageYrBlt        81
* GarageCond         81
* GarageType         81
* GarageFinish       81
* GarageQual         81

**Numerical**
* MasVnrArea          8
* LotFrontage       259


The null values are replaced :
* for categorical data -> with the most frequently used value of each feature
* for numerical data -> the mean value of each feature 


**For categorical data replace null values with the most frequently used value of each feature**

In [ ]:
#replace categorical null values with most frequent values present in each column
categorical_data = train_set.select_dtypes(['object']).columns
train_set[categorical_data] = train_set[categorical_data].fillna(train_set[categorical_data].mode().iloc[0])
train_set[categorical_data].mode()


**For numerical data replace null values with the mean value of each feature**

In [ ]:
#replace numerical null values with the mean value of each column

numerical_data = train_set.select_dtypes(['float64','int64']).columns
train_set[numerical_data] = train_set[numerical_data].fillna(train_set[numerical_data].mean())
train_set[numerical_data].mean()



In [ ]:
print(train_set.isnull().sum().sort_values(ascending = False))


In [ ]:
train_set.hist(figsize=(20,20), bins=20)
plt.show()

In [ ]:
train_set = train_set.drop(columns=['LowQualFinSF','PoolArea','MiscVal','3SsnPorch'])
print(len(train_set.columns))

In [ ]:
print(train_set.dtypes)

Most of the data are categorical. Before training the model, categorical data should be converted into numerical data

**Convert categorical data into numerical format**

In [ ]:
category_columns = train_set.select_dtypes(['object']).columns
print(category_columns)
train_set[category_columns] = train_set[category_columns].astype('category').apply(lambda x: x.cat.codes)

In [ ]:
float_columns = train_set.select_dtypes(['float64']).columns
print(float_columns)
train_set['LotFrontage'] =  pd.to_numeric(train_set['LotFrontage'], errors = 'coerce')
train_set['MasVnrArea'] =  pd.to_numeric(train_set['MasVnrArea'], errors = 'coerce')
train_set['GarageYrBlt'] =  pd.to_numeric(train_set['GarageYrBlt'], errors = 'coerce')
train_set['SalePrice'] =  pd.to_numeric(train_set['SalePrice'], errors = 'coerce')

train_set = train_set.astype('int64')


In [ ]:
print(train_set.dtypes)

**Sales Price**

In [ ]:
train_set['SalePrice'].describe()

In [ ]:
sns.displot(train_set['SalePrice'])

**Sales prices is left Skewwed. So it should be normalized**

In [ ]:
#Normalized Sales price 
#train_set.SalePrice = np.log(train_set.SalePrice)
#sns.displot(train_set['SalePrice'])


**Correlation of each attribue with price**

In [ ]:
correlation_matrix = train_set.corr()
correlation_matrix['SalePrice'].sort_values(ascending = False)

**Heat Map**

In [ ]:
#Heat map
correlation_num = 30
correlation_cols = correlation_matrix.nlargest(correlation_num,'SalePrice')['SalePrice'].index
correlation_mat_sales = np.corrcoef(train_set[correlation_cols].values.T)
sns.set(font_scale=1.25)
f, ax = plt.subplots(figsize=(12, 9))
hm = sns.heatmap(correlation_mat_sales, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 7}, yticklabels=correlation_cols.values, xticklabels=correlation_cols.values)
plt.show()

In [ ]:
y= train_set['SalePrice']
#x = train_set[['OverallQual','GrLivArea','GarageCars','GarageArea', 'TotalBsmtSF','1stFlrSF','FullBath','YearBuilt','YearRemodAdd','TotRmsAbvGrd']]
x= train_set.drop(columns=['SalePrice','Id'])
print(len(x.columns))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size = 0.3, random_state = 60,shuffle=True)
print(len(X_train))
print(len(X_test))
 

**Multiple Linear Regression**

In [ ]:

linear_model = LinearRegression()
linear_model.fit(X_train, Y_train)

#make predictions using linear regression
price_predict =(linear_model.predict(X_test))

#Measure Performance 
linear_mse = mean_squared_error(Y_test,price_predict)
print("Mean Squared error: ",linear_mse)
linear_SqMse = np.sqrt(linear_mse)
print("Root Mean Squared error: ", linear_SqMse)

print("Explain variance score for Linear Regression =", round(sm.explained_variance_score(Y_test,price_predict),2)) 
print("R2 score for Linear Regression =", round(sm.r2_score(Y_test, price_predict), 2))



In [ ]:
# Create a model with 50 decision trees
forest_model= RandomForestRegressor(n_estimators = 50, random_state = 42)

# Train the model with training data
forest_model.fit(X_train,Y_train)
#make predictions using random forest model
price_predict =(forest_model.predict(X_test))


forest_mse = mean_squared_error(Y_test,price_predict)
print("Mean Squared Error : " , forest_mse)
forest_SqMse = np.sqrt(forest_mse)
print("Root Mean Squared error: ", forest_SqMse)

print("Explain variance score for Random Forest Regression =", round(sm.explained_variance_score(Y_test,price_predict), 2)) 
print("R2 score for Random Forest Regression =", round(sm.r2_score(Y_test, price_predict), 2))



**Test Set**

In [ ]:
test_set.head()

In [ ]:
test_set.dtypes

In [ ]:
test_set.isnull().sum().sort_values(ascending = False)

In [ ]:

test_set=test_set.drop(columns=['PoolQC', 'MiscFeature', 'Alley', 'Fence','LowQualFinSF','PoolArea','MiscVal','3SsnPorch'])
test_set.describe()

In [ ]:
#replace categorical null values with most frequent values present in each column
categorical_data = test_set.select_dtypes(['object']).columns
test_set[categorical_data] = test_set[categorical_data].fillna(test_set[categorical_data].mode().iloc[0])
test_set[categorical_data].mode()


In [ ]:
#replace numerical null values with the mean value of each column

numerical_data = test_set.select_dtypes(['float64','int64']).columns
test_set[numerical_data] = test_set[numerical_data].fillna(test_set[numerical_data].mean())
test_set[numerical_data].mean()




In [ ]:
test_set.isnull().sum().sort_values(ascending = False)

**convert categorical into numerical**

In [ ]:
category_columns = test_set.select_dtypes(['object']).columns
print(category_columns)
test_set[category_columns] = test_set[category_columns].astype('category').apply(lambda x: x.cat.codes)

**Convert float in to int**

In [ ]:
float_columns = test_set.select_dtypes(['float64']).columns
print(float_columns)
test_set['LotFrontage'] =  pd.to_numeric(test_set['LotFrontage'], errors = 'coerce')
test_set['MasVnrArea'] =  pd.to_numeric(test_set['MasVnrArea'], errors = 'coerce')
test_set['GarageYrBlt'] =  pd.to_numeric(test_set['GarageYrBlt'], errors = 'coerce')
test_set['BsmtFinSF1'] =  pd.to_numeric(test_set['BsmtFinSF1'], errors = 'coerce')
test_set['BsmtFinSF2'] =  pd.to_numeric(test_set['BsmtFinSF2'], errors = 'coerce')
test_set['BsmtUnfSF'] =  pd.to_numeric(test_set['BsmtUnfSF'], errors = 'coerce')
test_set['TotalBsmtSF'] =  pd.to_numeric(test_set['TotalBsmtSF'], errors = 'coerce')
test_set['BsmtFullBath'] =  pd.to_numeric(test_set['BsmtFullBath'], errors = 'coerce')
test_set['GarageCars'] =  pd.to_numeric(test_set['GarageCars'], errors = 'coerce')
test_set['GarageArea'] =  pd.to_numeric(test_set['GarageArea'], errors = 'coerce')

test_set = test_set.astype('int64')


In [ ]:
test_id = test_set['Id']
test_x= test_set.drop(columns=['Id'])
print(len(test_x.columns))

In [ ]:
y_prdict = forest_model.predict(test_x)

In [ ]:
#submission = pd.DataFrame(test_id)
submission = pd.DataFrame(list(zip(test_id, y_prdict)),
               columns =['Id', 'SalePrice'])
submission.head(20)

In [ ]:
submission.to_csv('submission.csv', index=False)